# 数据集构建

In [1]:
import numpy as np

def loadSimpData():
    datMat = np.matrix([[ 1. ,  2.1],
                        [ 2.0,  1.1],
                        [ 1.3,  1. ],
                        [ 1. ,  1. ],
                        [ 2. ,  1. ]])
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return datMat,classLabels

In [2]:
datMat,classLabels = loadSimpData()

In [3]:
datMat

matrix([[ 1. ,  2.1],
        [ 2. ,  1.1],
        [ 1.3,  1. ],
        [ 1. ,  1. ],
        [ 2. ,  1. ]])

In [4]:
classLabels

[1.0, 1.0, -1.0, -1.0, 1.0]

# 单层决策树生成函数

In [5]:
def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):
    retArray = np.ones((np.shape(dataMatrix)[0],1)) # 初始化retArray为1
    if threshIneq == 'lt':
        retArray[dataMatrix[:,dimen] <= threshVal] = -1.0 # 如果小于阈值,则赋值为-1
    else:
        retArray[dataMatrix[:,dimen] > threshVal] = -1.0 # 如果大于阈值,则赋值为-1
    return retArray

def buildStump(dataArr,classLabels,D):
    dataMatrix = np.mat(dataArr); labelMat = np.mat(classLabels).T
    m,n = np.shape(dataMatrix)
    numSteps = 10.0; bestStump = {}; bestClasEst = np.mat(np.zeros((m,1)))
    minError = float('inf') # 最小误差初始化为正无穷大
    for i in range(n): # 遍历所有特征
        rangeMin = dataMatrix[:,i].min(); rangeMax = dataMatrix[:,i].max() # 找到特征中最小的值和最大值
        stepSize = (rangeMax - rangeMin) / numSteps # 计算步长
        for j in range(-1, int(numSteps) + 1):                                     
            for inequal in ['lt', 'gt']: # 大于和小于的情况，均遍历。lt:less than，gt:greater than
                threshVal = (rangeMin + float(j) * stepSize) # 计算阈值
                predictedVals = stumpClassify(dataMatrix, i, threshVal, inequal) # 计算分类结果
                errArr = np.mat(np.ones((m,1))) # 初始化误差矩阵
                errArr[predictedVals == labelMat] = 0 # 分类正确的,赋值为0
                weightedError = D.T * errArr # 计算误差
                # print("split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError))
                if weightedError < minError:  # 找到误差最小的分类方式
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump, minError, bestClasEst

In [6]:
D = np.mat(np.ones((5, 1)) / 5)
buildStump(datMat, classLabels, D)

({'dim': 0, 'ineq': 'lt', 'thresh': 1.3}, matrix([[ 0.2]]), array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

# 基于单层决策树的Adaboost训练过程

In [7]:
def adaBoostTrainDS(dataArr, classLabels, numIt = 40):
    weakClassArr = []
    m = np.shape(dataArr)[0]
    D = np.mat(np.ones((m, 1)) / m) # 初始化权重
    aggClassEst = np.mat(np.zeros((m,1)))
    for i in range(numIt):
        bestStump, error, classEst = buildStump(dataArr, classLabels, D) # 构建单层决策树
        print("D:",D.T)
        alpha = float(0.5 * np.log((1.0 - error) / max(error, 1e-16))) # 计算弱学习算法权重alpha,使error不等于0,因为分母不能为0
        bestStump['alpha'] = alpha # 存储弱学习算法权重
        weakClassArr.append(bestStump) # 存储单层决策树
        print("classEst: ", classEst.T)
        expon = np.multiply(-1 * alpha * np.mat(classLabels).T, classEst) # 计算e的指数项
        D = np.multiply(D, np.exp(expon))                                      
        D = D / D.sum() #根 据样本权重公式，更新样本权重
        # 计算AdaBoost误差，当误差为0的时候，退出循环
        aggClassEst += alpha * classEst #计算类别估计累计值                                
        print("aggClassEst: ", aggClassEst.T)
        aggErrors = np.multiply(np.sign(aggClassEst) != np.mat(classLabels).T, np.ones((m,1))) #计算误差
        errorRate = aggErrors.sum() / m
        print("total error: ", errorRate)
        if errorRate == 0.0: 
            break # 误差为0，退出循环
    return weakClassArr, aggClassEst

In [8]:
classifierArray = adaBoostTrainDS(datMat, classLabels, 9)

D: [[ 0.2  0.2  0.2  0.2  0.2]]
classEst:  [[-1.  1. -1. -1.  1.]]
aggClassEst:  [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
total error:  0.2
D: [[ 0.5    0.125  0.125  0.125  0.125]]
classEst:  [[ 1.  1. -1. -1. -1.]]
aggClassEst:  [[ 0.27980789  1.66610226 -1.66610226 -1.66610226 -0.27980789]]
total error:  0.2
D: [[ 0.28571429  0.07142857  0.07142857  0.07142857  0.5       ]]
classEst:  [[ 1.  1.  1.  1.  1.]]
aggClassEst:  [[ 1.17568763  2.56198199 -0.77022252 -0.77022252  0.61607184]]
total error:  0.0


In [9]:
classifierArray

([{'alpha': 0.6931471805599453, 'dim': 0, 'ineq': 'lt', 'thresh': 1.3},
  {'alpha': 0.9729550745276565, 'dim': 1, 'ineq': 'lt', 'thresh': 1.0},
  {'alpha': 0.8958797346140273,
   'dim': 0,
   'ineq': 'lt',
   'thresh': 0.90000000000000002}],
 matrix([[ 1.17568763],
         [ 2.56198199],
         [-0.77022252],
         [-0.77022252],
         [ 0.61607184]]))

# Adaboost分类函数

In [10]:
def adaClassify(datToClass,classifierArr):
    dataMatrix = np.mat(datToClass)
    m = np.shape(dataMatrix)[0]
    aggClassEst = np.mat(np.zeros((m,1)))
    for i in range(len(classifierArr)): # 遍历所有分类器，进行分类
        classEst = stumpClassify(dataMatrix, classifierArr[i]['dim'], classifierArr[i]['thresh'], classifierArr[i]['ineq'])            
        aggClassEst += classifierArr[i]['alpha'] * classEst
        print(aggClassEst)
    return np.sign(aggClassEst)

In [11]:
datArr, labelArr = loadSimpData()
classifierArr, aggClassEst = adaBoostTrainDS(datArr, labelArr, 30)

D: [[ 0.2  0.2  0.2  0.2  0.2]]
classEst:  [[-1.  1. -1. -1.  1.]]
aggClassEst:  [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
total error:  0.2
D: [[ 0.5    0.125  0.125  0.125  0.125]]
classEst:  [[ 1.  1. -1. -1. -1.]]
aggClassEst:  [[ 0.27980789  1.66610226 -1.66610226 -1.66610226 -0.27980789]]
total error:  0.2
D: [[ 0.28571429  0.07142857  0.07142857  0.07142857  0.5       ]]
classEst:  [[ 1.  1.  1.  1.  1.]]
aggClassEst:  [[ 1.17568763  2.56198199 -0.77022252 -0.77022252  0.61607184]]
total error:  0.0


In [12]:
adaClassify([0, 0], classifierArr)

[[-0.69314718]]
[[-1.66610226]]
[[-2.56198199]]


matrix([[-1.]])

In [13]:
adaClassify([[5, 5], [0, 0]], classifierArr)

[[ 0.69314718]
 [-0.69314718]]
[[ 1.66610226]
 [-1.66610226]]
[[ 2.56198199]
 [-2.56198199]]


matrix([[ 1.],
        [-1.]])

# 在一个难数据集上应用Adaboost

## 自适应数据加载函数

In [14]:
def loadDataSet(filename):
    numFeat = len(open(filename).readline().split('\t')) # 读取数据，以制表符分隔
    dataMat = []
    labelMat = []
    fr = open(filename)
    for line in fr.readlines():
        lineArr = []
        curLine = line.strip().split('\t')
        for i in range(numFeat - 1):
            lineArr.append(float(curLine[i]))
        dataMat.append(lineArr)
        labelMat.append(float(curLine[-1]))
    return dataMat, labelMat

In [15]:
datArr, labelArr = loadDataSet('./horseColicTraining2.txt')
classifierArray, aggClassEstArr = adaBoostTrainDS(datArr, labelArr, 10)

D: [[ 0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.00334448  0.00334448  0.00334448  0.00334448
   0.00334448  0.00334448  0.003344

   0.46166238 -0.46166238 -0.46166238  0.46166238  0.46166238]]
total error:  0.284280936455
D: [[ 0.00233645  0.00588235  0.00233645  0.00588235  0.00588235  0.00233645
   0.00233645  0.00588235  0.00233645  0.00588235  0.00233645  0.00233645
   0.00233645  0.00588235  0.00233645  0.00233645  0.00233645  0.00233645
   0.00233645  0.00233645  0.00588235  0.00233645  0.00233645  0.00233645
   0.00233645  0.00233645  0.00233645  0.00233645  0.00588235  0.00233645
   0.00233645  0.00588235  0.00233645  0.00233645  0.00588235  0.00233645
   0.00588235  0.00588235  0.00233645  0.00588235  0.00233645  0.00233645
   0.00233645  0.00233645  0.00233645  0.00233645  0.00588235  0.00233645
   0.00588235  0.00233645  0.00233645  0.00588235  0.00233645  0.00233645
   0.00588235  0.00588235  0.00233645  0.00233645  0.00233645  0.00233645
   0.00233645  0.00233645  0.00588235  0.00588235  0.00233645  0.00233645
   0.00233645  0.00233645  0.00233645  0.00233645  0.00588235  0.00233645
   0.00588235  0

classEst:  [[ 1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
   1.  1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.
  -1.  1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
   1.  1. -1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1. -1.  1.
  -1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.
  -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.
   1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.
   1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1. -1. -1.  1. -1.  1.  1.  1.
  -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1. -1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
  -1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1. -1.  1.  1.
   1.  1.  1.  1.  1.  1.  

In [16]:
testArr, testLabelArr = loadDataSet('./horseColicTest2.txt')
prediction10 = adaClassify(testArr, classifierArray)

[[ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166

In [17]:
errArr = np.mat(np.ones((67, 1)))
errArr[prediction10 != np.mat(testLabelArr).T].sum()

16.0